In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("ARK_API_KEY")
MODEL=os.getenv("MODEL")
BASE_URL=os.getenv("BASE_URL")



In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import before_model, after_model, AgentState
from langchain.chat_models import init_chat_model
from langgraph.runtime import Runtime
model=init_chat_model(
    model=MODEL,
    model_provider="openai",
    api_key=API_KEY,
    base_url=BASE_URL,
)

In [15]:
import asyncio
from langsmith import traceable
# @traceable(run_type="llm")
# def process_sequentially(queries:list):
#     res=[]
#     for query in queries:
#         response=model.invoke(query)
#         res.append(response)
#     return res
# @traceable(run_type="llm")
# def process_batch(queries:list):
#     return model.batch(queries)
@traceable(run_type="llm")
async def process_async(queries:list):
    tasks=[model.ainvoke(query) for query in queries]
    results=await asyncio.gather(*tasks)
    return [r.content for r in results]

In [16]:
import time
queries = [f"问题{i}" for i in range(10)]

# print("串行处理:")
# start = time.time()
# process_sequentially(queries)
# print(f"耗时: {time.time() - start:.2f}秒\n")

# print("批处理:")
# start = time.time()
# process_batch(queries)
# print(f"耗时: {time.time() - start:.2f}秒\n")

print("异步处理:")
start = time.time()
# In Colab, an event loop is usually running already. Direct `await` is preferred.
await process_async(queries)
print(f"耗时: {time.time() - start:.2f}秒")

异步处理:
耗时: 5.06秒
